In [1]:
import numpy as np
import pandas as pd
import time
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from datetime import datetime
import scipy.special as special
import os
import csv
from numpy.random import RandomState

# Fixed Seed for Repeatability
seed = 5
np.random.seed(seed)
c1=4 # tacc = c1 tFWHM: c1 = 1.3 from Fuchs
c2=0.74 # intensity exponent in hot electron fraction f: c2 = 0.74 from Fuchs
theta=25 # Divergence Angle theta: theta = 25 deg from Fuchs
const_f = False # Don't assume f is fixed at 0.5
noise=30 # Noise level in percent

In [2]:
def calc_laser_energy(I0, w0, tFWHM):
    return (np.pi/2)*I0*w0**2 * tFWHM # For a Sine-Squared Pulse
    #return (np.pi/2)*I0*w0**2 * tFWHM * np.sqrt(np.pi / (4*np.log(2))) # For a Gaussian Shaped Pulse

# integrate dN/dE from eMin to eMax from Eq. (2) in Fuchs Paper
def calc_N_between(ne, cs, tacc, S, Tp, eMin=0, eMax=1):
    xmin = np.sqrt(2*eMin/Tp)
    xmax = np.sqrt(2*eMax/Tp)
    return ne*cs*tacc*S*(np.exp(-xmin) - np.exp(-xmax))

# integrate dN/dE * E from eMin to eMax from Eq. (2) in Fuchs Paper
def calc_E_between(ne, cs, tacc, S, Tp, eMin=0, eMax=1):
    xmin = np.sqrt(2*eMin/Tp)
    xmax = np.sqrt(2*eMax/Tp)
    return ne*cs*tacc*S*Tp/2*(np.exp(-xmin)*(2 + xmin*(2 + xmin)) - np.exp(-xmax)*(2 + xmax*(2 + xmax)))

# Calculate Max proton energy in terms of normalized acceleration time and hot temperature Tp from Eq. (1) in Fuchs Paper
def calc_max_E(omega_pi, tacc, Tp):
    tp = omega_pi*tacc / np.sqrt(2*np.exp(1))
    return 2*Tp*np.log(tp + np.sqrt(tp**2 + 1))**2

def gaussian_noise(energy_list, pct_noise, random_seed = False):
    prng = RandomState(seed)
    α = pct_noise/100 # Fraction instead of percent
    noisy_energy_list = prng.normal(energy_list, energy_list*α)
    for i in range(len(noisy_energy_list)):
        while(noisy_energy_list[i] < 0):
            print('found (-) energy value at i = {} during dataset generation, resampling now ...'.format(i))
            noisy_energy_list[i] = prng.normal(energy_list[i], energy_list[i]*α)
    return noisy_energy_list

def log_gaussian_noise(energy_list, pct_noise, random_seed = False):
    if random_seed:
        prng = RandomState()
    else: 
        prng = RandomState(seed)
    α = pct_noise/100 # Fraction instead of percent
    noisy_energy_list = np.zeros(len(energy_list))
    for i in range(len(noisy_energy_list)):
        mu = np.log(energy_list[i]/np.sqrt(1+α**2))
        σ = np.sqrt(np.log(1+α**2))
        noisy_energy_list[i] = prng.lognormal(mu, σ)
    return noisy_energy_list
    
def fuchs_model(I0, z, d, w0 = 1.5e-6, lmda = 0.8e-6, tFWHM = 40.0e-15, c1 = 1.3, c2 = 0.74, theta = 25, pct_noise = 0, const_f = False):
    c = 2.998e8
    m= 9.109e-31
    e=1.602e-19
    mi=1.673e-27
    Zi=1
    eps0=8.854e-12
    laser_energy = calc_laser_energy(I0, w0, tFWHM)
    
    omega = 2*np.pi*c/lmda
    zR = np.pi * w0**2 / lmda
    theta_rad = theta*np.pi/180
    Iz = I0 / (1 + (z/zR)**2)
    w = w0*np.sqrt(1 + (z/zR)**2)
    E0 = np.sqrt(2*Iz / (c*eps0))
    a0 = (e*E0)/(m*omega*c)
    gamma = np.sqrt(1+a0**2)
    Tp = m*c**2*(gamma - 1)
    # T_Wilks == Tp is the Hot Electron Temperature calculated in an equivalent way that I did it
    # T_Wilks = 0.511 * (np.sqrt(1+(Iz*1e-4)*(lmda*1e6)**2/1.37E18) -1)
    if const_f:
        f = 0.5
    else:
        f = np.minimum(1.2e-15 * (Iz*1e-4)**c2, 0.5)
    Ne = f*laser_energy / Tp
    r0 = w * np.sqrt(2*np.log(2))/2
    S = np.pi*(r0 + d*np.tan(theta_rad))**2
    ne = Ne / (S*c*tFWHM)
    omega_pi = np.sqrt(Zi * e**2 *ne / (mi*eps0))
    tacc = c1 * tFWHM

    max_proton_energy = calc_max_E(omega_pi, tacc, Tp)

    cs = np.sqrt(Zi*Tp/mi) # Sound Speed
    num_protons = calc_N_between(ne, cs, tacc, S, Tp, eMin=0, eMax=max_proton_energy)
    total_proton_energy = calc_E_between(ne, cs, tacc, S, Tp, eMin=0, eMax=max_proton_energy)
    #num_protons = calc_N_above_eMin(ne, cs, tacc, S, Tp, eMin=0) - calc_N_above_eMin(ne, cs, tacc, S, Tp, eMin=max_proton_energy)
    #total_proton_energy = calc_E_below_eMin(ne, cs, tacc, S, Tp, eMin=eMax) - calc_E_below_eMin(ne, cs, tacc, S, Tp, eMin=0)
    average_proton_energy = total_proton_energy/num_protons

    # Convert Energies to MeV
    max_proton_energy_MeV = max_proton_energy / (1.6e-13)
    total_proton_energy_MeV = total_proton_energy / (1.6e-13)
    average_proton_energy_MeV = average_proton_energy / (1.6e-13)
    
    # Laser to Proton Energy Conversion Ratio
    laser_conversion_efficiency = total_proton_energy / laser_energy
    
    # Add Log Gaussian Noise
    noisy_max_proton_energy_MeV = log_gaussian_noise(max_proton_energy_MeV, pct_noise)
    noisy_total_proton_energy_MeV = log_gaussian_noise(total_proton_energy_MeV, pct_noise)
    noisy_average_proton_energy_MeV = log_gaussian_noise(average_proton_energy_MeV, pct_noise)
    
    return (noisy_max_proton_energy_MeV, noisy_total_proton_energy_MeV, noisy_average_proton_energy_MeV)

In [3]:
def model(X):
    return fuchs_model(X[:, 0]*1e4, X[:, 2]*1e-6, X[:, 1]*1e-6, c1=c1, c2=c2, theta=theta, pct_noise=noise, const_f=const_f)

def generate_random_points(bounds, n):
    np.random.seed(0)
    points = []
    for bound in bounds:
        points.append(np.random.uniform(bound[0], bound[1], n))
    return np.array(points).transpose()

In [4]:
bounds = [(1e19, 1e19), (0.0, 10.0), (0, 10.0)]
n_points = 2000
points = generate_random_points(bounds, n_points)
Emax, Etot, Eavg = model(points)
print(Emax[0])
output_df = pd.DataFrame(columns=['Intensity', 'Thickness', 'Offset', 'E Max', 'E Tot', 'E Avg'])
output_df['Intensity'] = points[:, 0]
output_df['Thickness'] = points[:, 1]
output_df['Offset'] = points[:, 2]
output_df['E Max'] = Emax
output_df['E Tot'] = Etot
output_df['E Avg'] = Eavg
output_df.head()

0.22719896544427706


,Intensity,Thickness,Offset,E Max,E Tot,E Avg
0,1.000000e+19,8.115185,2.926420,0.227199,6.099050e+07,0.061202
1,1.000000e+19,4.760840,5.665183,0.299769,8.212706e+07,0.073554
2,1.000000e+19,5.231560,1.374144,0.824788,2.541070e+08,0.203419
3,1.000000e+19,2.505206,3.497122,0.755437,2.279427e+08,0.158600
4,1.000000e+19,6.050430,0.532164,0.348687,1.055944e+08,0.088602


In [5]:
output_df.to_csv('predictions_dfs/fuchs_{}.csv'.format(n_points), index=False)